토크나이징이 되어있는 데이터를 불러드립니다. 포멧을 확인합니다. 

In [2]:
movie_reviews = './data/merged_movie_comments_tsoynlp.txt'
with open(movie_reviews, encoding='utf-8') as f:
    for _ in range(3):
        print(next(f).strip())

72523	명불허전	1
72523	왠지 고사 피의 중간 고사 보다 재미 가 없을 듯해요 만약 보게 된다 면 실망 할듯	1
72523	티아라 사랑 해 ㅜ	10


적절한 load_comments() 함수를 만들어 데이터를 로딩합니다. 3,280,685 개의 텍스트와 평점을 로딩하였습니다. 

In [2]:
def load_comments(fname):
    with open(fname, encoding='utf-8') as f:
        docs = [doc.strip().split('\t') for doc in f]
    docs = [(doc[1], int(doc[2])) for doc in docs if len(doc) == 3]
    texts, scores = zip(*docs)
    return texts, scores

texts, scores = load_comments(movie_reviews)
len(texts), len(scores)

(3280685, 3280685)

단어의 min frequency cutting을 하기 위해 먼저 texts에 있는 모든 단어들의 빈도수를 확인합니다. 

    [word for text in texts for word in text.split()]
    
위 list 는 texts의 각 text마다 split()을 한 뒤, 그 단어들을 하나의 list로 flatten 하는 코드입니다. 

    ['a b', 'c d e'] --> ['a', 'b', 'c', 'd', 'e']

처럼 결과가 나옵니다. 

그런데, list가 아니라 맨 앞과 뒤를 [, ] 대신 (, )를 쓰면 generator가 만들어집니다. 

    (word for text in texts for word in text.split())
    
generator는 한마디로 말하면 (정확한 표현이 아닙니다), 메모리에 필요한 값만 올려서 쓰는 list라고 생각하셔도 됩니다. 이미 있는 texts라는 list를 flatten한 새로운 list는 만들지 않고 쓰겠다는 의미입니다. 

여러분은 이미 generator를 쓰시고 계십니다. open(fname) 역시 한 줄을 읽고 이를 yield 하는 generator입니다. 

또한 psutil이라는 package를 쓰면 현재 프로세스 (주피터 파일, 혹은 파이썬 파일)이 사용하는 메모리의 양이 출력됩니다. soynlp.utils에 get_process_memory() 라는 함수로 wrapping 해두었습니다. 필요하실 때 쓰십시요. 

In [3]:
from collections import Counter
word_counter = Counter((word for text in texts for word in text.split()))

from soynlp.utils import get_process_memory
print('used memory = %.3f Gb' % get_process_memory())

used memory = 1.346 Gb


빈도수 기준 상위 20개의 단어를 출력합니다. 

In [4]:
sorted(word_counter.items(), key=lambda x:x[1], reverse=True)[:20]

[('영화', 1412516),
 ('이', 764006),
 ('관람객', 585858),
 ('는', 459876),
 ('가', 438771),
 ('도', 418073),
 ('의', 403943),
 ('다', 381746),
 ('재밌', 370724),
 ('재미', 344634),
 ('너무', 335529),
 ('ㅋㅋ', 321284),
 ('정말', 297962),
 ('고', 294644),
 ('을', 270826),
 ('에', 266720),
 ('한', 263385),
 ('를', 263311),
 ('연기', 255673),
 ('최고', 254291)]

min_count = 50으로 frequency cutting 을 할 때, 단어의 개수의 차이를 확인합니다. 총 22,451개의 단어가 존재합니다. 

In [5]:
n_words_before_pruning = len(word_counter)

min_count = 50
word_dictionary = {word:freq for word,freq in word_counter.items() if freq >= min_count}
n_words_after_pruning  = len(word_dictionary)

print('%d --> %d' % (n_words_before_pruning, n_words_after_pruning))

341744 --> 22451


평점 별로도 문서의 개수를 확인합니다. 유명한 영화들이다보니 10점이 압도적으로 많음을 알 수 있습니다. 애초에 좋은 영화가 아니면 사람들이 많이 보지 않았겠죠? 

In [6]:
for score, score_freq in Counter(scores).items():
    print('score = %d: (%d, %.3f perc)' % (score, score_freq, 100*score_freq/len(scores)))

score = 1: (320898, 9.781 perc)
score = 2: (34351, 1.047 perc)
score = 3: (35580, 1.085 perc)
score = 4: (39742, 1.211 perc)
score = 5: (78250, 2.385 perc)
score = 6: (95834, 2.921 perc)
score = 7: (149618, 4.561 perc)
score = 8: (268622, 8.188 perc)
score = 9: (344905, 10.513 perc)
score = 10: (1912885, 58.307 perc)


min count cutting을 하다보니 희귀한 단어로만 이뤄진 문장도 존재할 수 있습니다. 이 경우에는 zero vector가 만들어질겁니다. 이를 방지하기 위해서 학습용 데이터를 따로 만들겠습니다. 

그리고 binary classification을 하기 위해서 3점 이하의 영화평을 negative class, 9점 이상의 영화평을 positive class로 만들겠습니다. 

    words = [word for word in text.split() if word in word_dictionary]
    if not words:
        continue
        
위 부분은 text를 split() 한 다음, 각 단어가 word_dictionary (min_count >= 50인 단어 집합)에 등록되었는지 확인합니다. 만약 words가 empty list이면 학습데이터에 추가하지 않고 다음 text를 살펴봅니다. 

    train_label.append(1 if score > 8 else -1)
    
위 부분은 평점이 9 혹은 10점이면 1이라는 label을, 그렇지 않다면 -1이라는 label을 train_label에 추가합니다. 어자피 [4, 8]점의 영화평들은 아래의 구문에 의하여 존재하지 않으니까요. 

    if 4 <= score <= 8:
        continue
        
여기서 한 가지 조금 특이하게 train_texts를 만들었습니다. words는 list of str입니다. 그렇기 때문에 train_texts는 list of list of str입니다. 

In [7]:
train_texts = []
train_label = []

for text, score in zip(texts, scores):
    if 4 <= score <= 8:
        continue
        
    words = [word for word in text.split() if word in word_dictionary]
    if not words:
        continue
    
    train_texts.append(words)
    train_label.append(1 if score > 8 else -1)

print('train data: %d --> %d' % (len(texts), len(train_texts)))
for label, label_freq in Counter(train_label).items():
    print('label = %d: (%d, %.3f perc)' % (label, label_freq, 100*label_freq/len(train_label)))

train data: 3280685 --> 2641694
label = 1: (2252779, 85.278 perc)
label = -1: (388915, 14.722 perc)


CountVectorizer를 이용하여 train_x, term frequency matrix를 만들겠습니다. 이미 train_texts는 토크나이징이 완료되어 있습니다. CountVectorizer에서 lowercase=False로 만들고, tokenizer를 lambda x:x로 두면 아무런 처리를 하지 않은 체 train_texts를 입력할 수 있습니다. 

sparse matrix에서 이용한 단어의 개수가 22,451개로 word_dictionary의 개수와 같습니다. min_df, max_df를 설정하지 않았기 때문에 빈도수가 50이상인 모든 단어들을 이용하여 sparse matrix를 만들었습니다. 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=lambda x:x, lowercase=False)
train_x = vectorizer.fit_transform(train_texts)
train_x.shape

(2641694, 22451)

이전의 실습처럼 vectorizer.vocabulary\_로부터 vocablist를 만들 수 있습니다. 

In [13]:
# with open('vocablist', 'w', encoding='utf-8') as f:
#     for word, _ in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1]):
#         f.write('%s\n' % word)
vocablist = [word for word, _ in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [52]:
vocablist[100:105]

['1주일', '1차', '1초', '1초도', '1탄']

여기까지하여 만들어둔 데이터를 data/ 폴더에 저장해 두었습니다. 

In [10]:
SAVE = False
if SAVE:
    import pickle
    with open('./tmp/sentiment_x.pkl', 'wb') as f:
        pickle.dump(train_x, f)

    with open('./tmp/sentiment_y.pkl', 'wb') as f:
        pickle.dump(train_label, f)
        
    with open('./tmp/sentiment_vocab.pkl', 'wb') as f:
        pickle.dump(vocablist, f)

L2 regularization이 Logistic Regression의 기본모델입니다. 물론 L2 cost를 C = 0으로 주면 전혀 regularization이 일어나지 않습니다만, 많은 경우 overfitting이 일어나니 L2를 이용하시기 바랍니다. default cost는 C=1입니다. 

    logistic_l2 = LogisticRegression(C = 1)

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_l2 = LogisticRegression()
logistic_l2.fit(train_x, train_label)

임의의 문장 30, 50, 64323에 대하여 실제 데이터와 레이블, 그리고 logistic regression이 예측하는 class probability / class label을 출력합니다. 

    logistic_l2.predict_proba(text_x)
    
Logistic Regression은 predict 의 input으로 matrix가 들어올 것을, 동시에 여러 개의 queries가 들어올 것을 가정하고 만든 알고리즘입니다. 하나의 query를 prediction 할 때에는 그 결과값을

    logistic_l2.predict_proba(train_x[idx,:])[0]
    
과 같이 하여야 됩니다. 

In [38]:
for idx in [30, 50, 64323]:
    print('text: %s\nlabel: %d\n' % (train_texts[idx], train_label[idx]))
    
    prob = logistic_l2.predict_proba(train_x[idx,:])[0]
    print('class prob: (negative= %.3f, positive= %.3f' 
          % tuple(prob))
    
    pred_label = logistic_l2.predict(train_x[idx,:])[0]
    print('class = %s' % 'positive' if pred_label == 1 else 'negative')
    print('\n%s\n' % ('-'*50))

text: ['지연', '누나', '보러', '왓는데']
label: 1

class prob: (negative= 0.142, positive= 0.858
class = positive

--------------------------------------------------

text: ['솔직', '희', '진짜', '별로', '재미', '없음', '지연', '누나', '땜시', '1개', '반', '준거', '임']
label: -1

class prob: (negative= 0.985, positive= 0.015
negative

--------------------------------------------------

text: ['진짜', '엄청', '나다', '이', '영화', '를', '볼수있', '다니', '행복', '지금', '12', '살', '미만', '인', '아이', '들이', '불쌍', '하다', '이런', '영화', '를', '영화', '관', '에서', '못보', '다니', '진짜', '엄청', '난', '영화', '다', '재밌', '냐고', '재미', '정도', '가', '아니', '다']
label: 1

class prob: (negative= 0.001, positive= 0.999
class = positive

--------------------------------------------------



Lasso penalty를 Logistic Regression에 넣어 보겠습니다. Regularization을 강하게 주기 위하여 C = 0.1로 주었습니다. 학습하는 부분은 동일합니다. 

In [ ]:
logistic_l1 = LogisticRegression(C=0.1, penalty='l1')
logistic_l1.fit(train_x, train_label)

위와 동일한 test를 해보았습니다. 결과는 비슷합니다. 

In [40]:
for idx in [30, 50, 64323]:
    print('text: %s\nlabel: %d\n' % (train_texts[idx], train_label[idx]))
    
    prob = logistic_l1.predict_proba(train_x[idx,:])[0]
    print('class prob: (negative= %.3f, positive= %.3f' 
          % tuple(prob))
    
    pred_label = logistic_l1.predict(train_x[idx,:])[0]
    print('class = %s' % 'positive' if pred_label == 1 else 'negative')
    print('\n%s\n' % ('-'*50))

text: ['지연', '누나', '보러', '왓는데']
label: 1

class prob: (negative= 0.140, positive= 0.860
class = positive

--------------------------------------------------

text: ['솔직', '희', '진짜', '별로', '재미', '없음', '지연', '누나', '땜시', '1개', '반', '준거', '임']
label: -1

class prob: (negative= 0.980, positive= 0.020
negative

--------------------------------------------------

text: ['진짜', '엄청', '나다', '이', '영화', '를', '볼수있', '다니', '행복', '지금', '12', '살', '미만', '인', '아이', '들이', '불쌍', '하다', '이런', '영화', '를', '영화', '관', '에서', '못보', '다니', '진짜', '엄청', '난', '영화', '다', '재밌', '냐고', '재미', '정도', '가', '아니', '다']
label: 1

class prob: (negative= 0.001, positive= 0.999
class = positive

--------------------------------------------------



Logistic Regression의 Coefficients를 뜯어보겠습니다. 

    LogisticRegression.coef_
    
는 (1 by n_vocab)의 ndarray입니다. 우리가 binary classification을 하였기 때문입니다. multi class logistic regression을 하면 (n_class by n_vocab)의 coefficient matrix가 만들어집니다. 

학습하는 방법은 위와 동일합니다. LogisticRegression이 train_label의 unique label의 개수를 확인한 뒤, multi-class classification이면 알아서 Softmax regression으로 바꿔둡니다. 

In [42]:
print(type(logistic_l1.coef_))
print(logistic_l1.coef_.shape)

<class 'numpy.ndarray'>
(1, 22451)


이 ndarray를 list로 풀어보겠습니다. (1 by n_vocab)이기 때문에 coefficients[0]을 하면 positive class를 예측하는 각 단어의 coefficient가 출력됩니다. 

In [45]:
coefficients = logistic_l1.coef_.tolist()
coefficients[0][:5]

[-0.29573326401702005,
 0.06048288324875791,
 0.3599387791174224,
 0.06875783921374953,
 -0.055198722191172445]

Positive class에 가까운 단어들을 살펴보겠습니다. coefficient의 enumerate를 sorting하면 각 단어의 coefficient가 큰 순서대로 정렬되어 출력됩니다. 

In [49]:
sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
sorted_coefficients[:5]

[(20531, 3.6810552915713934),
 (3702, 3.173158488667234),
 (6710, 2.7119258656919767),
 (2762, 2.702309178468442),
 (17070, 2.696817974616877)]

긍정적인 영화평에서 자주 나오는 단어들 상위 50개를 출력하면 아래와 같습니다. 

In [50]:
for word, coef in sorted_coefficients[:50]:
    print('%s (%.3f)' % (vocablist[word], coef))

틈이 (3.681)
꿀잼 (3.173)
또보고싶 (2.712)
굿 (2.702)
재밋어요 (2.697)
시키지 (2.678)
쵝오 (2.584)
굳 (2.538)
여운이 (2.497)
존잼 (2.494)
최고 (2.468)
짱임 (2.468)
흠잡을 (2.436)
알이즈웰 (2.429)
완벽 (2.343)
굿굿 (2.341)
대박 (2.311)
짱 (2.279)
잼씀 (2.245)
잊을수 (2.228)
있구만 (2.223)
시간가는줄 (2.217)
졸잼 (2.201)
안아까운 (2.199)
안아까 (2.197)
최곱니다 (2.169)
Good (2.165)
꼭보 (2.123)
짱이 (2.106)
재밋엇어요 (2.097)
안보면 (2.041)
강추 (2.020)
따뜻 (2.004)
뗄수 (2.003)
이정도면 (1.987)
굳굳 (1.979)
은위 (1.959)
짱짱 (1.944)
먹먹 (1.943)
있던데 (1.941)
완벽한 (1.937)
잼나 (1.937)
만으로도 (1.934)
만족 (1.925)
슬퍼요 (1.913)
아깝지 (1.909)
점줌 (1.906)
낮지 (1.905)
할틈 (1.903)
좋네요 (1.880)


부정적인 영화평에서 자주 나오는 단어들 상위 50개를 출력하면 아래와 같습니다. 

In [51]:
for word, coef in sorted_coefficients[-50:]:
    print('%s (%.3f)' % (vocablist[word], coef))

O기 (-2.135)
시청자 (-2.138)
총기를 (-2.154)
거품이 (-2.155)
컸나 (-2.155)
이하도 (-2.177)
딴걸 (-2.194)
졸았 (-2.194)
비추 (-2.197)
이하 (-2.201)
숙면 (-2.202)
싶지 (-2.238)
아까 (-2.242)
졸려 (-2.251)
그다지 (-2.254)
할인카드 (-2.257)
망작 (-2.308)
안볼란다 (-2.312)
잤어요 (-2.313)
음모론 (-2.323)
이딴 (-2.332)
그닥 (-2.351)
나가고 (-2.352)
퇴보 (-2.352)
잤다 (-2.354)
김일병 (-2.374)
엉망 (-2.385)
실망 (-2.390)
점준 (-2.408)
빵점 (-2.421)
거품 (-2.427)
웃대 (-2.448)
이딴게 (-2.508)
ㅁㅈㅎ (-2.558)
짜집기 (-2.569)
차라리 (-2.572)
긴급조치 (-2.620)
별루 (-2.623)
불면증 (-2.670)
긴급조치19호 (-2.770)
졸작 (-2.790)
과대평가 (-2.965)
방어율 (-3.088)
내돈 (-3.097)
핵노잼 (-3.166)
노잼 (-3.224)
최악이다 (-3.337)
돈아까 (-3.412)
최악의 (-4.026)
최악 (-4.100)
